In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.experimental import enable_iterative_imputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.impute import IterativeImputer
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold
from sklearn.svm import OneClassSVM

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,precision_recall_curve, \
roc_auc_score,roc_curve,recall_score,classification_report, f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.svm import OneClassSVM
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedShuffleSplit
from lightgbm import LGBMClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import imblearn
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
from imblearn import pipeline

In [2]:
df = pd.read_csv('../input/iba-ml1-mid-project/train.csv')
df['credit_line_utilization'] = df['credit_line_utilization'].str.replace(',', '.').astype(float)
columns = df.iloc[:, 1:-1].columns
columns

Index(['age', 'number_dependent_family_members', 'monthly_income',
       'number_of_credit_lines', 'real_estate_loans',
       'ratio_debt_payment_to_income', 'credit_line_utilization',
       'number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more'],
      dtype='object')

In [3]:
X, y = df[columns], df.iloc[:, -1]
Counter(y)

Counter({0: 67148, 1: 5013})

In [4]:
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X, y)

In [24]:
X_imputed = IterativeImputer().fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns

In [33]:
ee = IsolationForest()
yhat = ee.fit_predict(X_imputed)
mask = yhat != -1
X_out, y_out = X_imputed[mask], y_rus[mask]

In [34]:
Counter(y_out)

Counter({0: 4904, 1: 4611})

In [ ]:
roc = []
for _ in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0) 
    
    pipez = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipez.fit(X_train, y_train)
    y_predicted=pipez.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
#     recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
# print(np.mean(recall))

In [28]:
roc = []
for _ in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_out, y_out, test_size=.3, random_state=0) 
    
    pipe7 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe7.fit(X_train, y_train)
    y_predicted=pipe7.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
#     recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
# print(np.mean(recall))

0.8369868468271393


In [39]:
roc = []
for _ in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_out, y_out, test_size=.3, random_state=0) 
    
    pipe7 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', GradientBoostingClassifier())
    ])
    pipe7.fit(X_train, y_train)
    y_predicted=pipe7.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
#     recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
# print(np.mean(recall))

0.8300302917684256


In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe8 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(RandomForestClassifier(n_estimators=200), n_estimators=20))
    ])
    pipe8.fit(X_train, y_train)
    y_predicted=pipe8.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe8 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=200)))
    ])
    pipe8.fit(X_train, y_train)
    y_predicted=pipe8.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [29]:
sss = StratifiedShuffleSplit()
roc = []
for train_idx, test_idx in sss.split(X_out, y_out):
    X_train, X_test = X_out.iloc[train_idx], X_out.iloc[test_idx]
    y_train, y_test = y_out.iloc[train_idx], y_out.iloc[test_idx]
    
    pipe82 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe82.fit(X_train, y_train)
    y_predicted=pipe82.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.818972420380871


In [30]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_out, y_out):
    X_train, X_test = X_out.iloc[train_idx], X_out.iloc[test_idx]
    y_train, y_test = y_out.iloc[train_idx], y_out.iloc[test_idx]
    
    pipe82 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', CatBoostClassifier())
    ])
    pipe82.fit(X_train, y_train)
    y_predicted=pipe82.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

Learning rate set to 0.024943
0:	learn: 0.6829000	total: 64ms	remaining: 1m 3s
1:	learn: 0.6727626	total: 70.6ms	remaining: 35.2s
2:	learn: 0.6635703	total: 77.7ms	remaining: 25.8s
3:	learn: 0.6547880	total: 83.6ms	remaining: 20.8s
4:	learn: 0.6468465	total: 89.5ms	remaining: 17.8s
5:	learn: 0.6387263	total: 96.2ms	remaining: 15.9s
6:	learn: 0.6313906	total: 102ms	remaining: 14.4s
7:	learn: 0.6242150	total: 107ms	remaining: 13.2s
8:	learn: 0.6179706	total: 112ms	remaining: 12.4s
9:	learn: 0.6122977	total: 118ms	remaining: 11.6s
10:	learn: 0.6062174	total: 123ms	remaining: 11.1s
11:	learn: 0.6006447	total: 128ms	remaining: 10.6s
12:	learn: 0.5950428	total: 134ms	remaining: 10.1s
13:	learn: 0.5903141	total: 139ms	remaining: 9.8s
14:	learn: 0.5858321	total: 145ms	remaining: 9.54s
15:	learn: 0.5814448	total: 151ms	remaining: 9.28s
16:	learn: 0.5772429	total: 156ms	remaining: 9.04s
17:	learn: 0.5738210	total: 162ms	remaining: 8.84s
18:	learn: 0.5698677	total: 168ms	remaining: 8.65s
19:	lear

179:	learn: 0.4669799	total: 1.17s	remaining: 5.32s
180:	learn: 0.4668604	total: 1.17s	remaining: 5.32s
181:	learn: 0.4667162	total: 1.18s	remaining: 5.3s
182:	learn: 0.4665608	total: 1.19s	remaining: 5.29s
183:	learn: 0.4663053	total: 1.19s	remaining: 5.28s
184:	learn: 0.4661370	total: 1.2s	remaining: 5.27s
185:	learn: 0.4660108	total: 1.2s	remaining: 5.26s
186:	learn: 0.4658000	total: 1.21s	remaining: 5.26s
187:	learn: 0.4656962	total: 1.22s	remaining: 5.25s
188:	learn: 0.4654957	total: 1.22s	remaining: 5.25s
189:	learn: 0.4653440	total: 1.23s	remaining: 5.24s
190:	learn: 0.4652431	total: 1.23s	remaining: 5.23s
191:	learn: 0.4651041	total: 1.24s	remaining: 5.22s
192:	learn: 0.4648764	total: 1.25s	remaining: 5.21s
193:	learn: 0.4646755	total: 1.25s	remaining: 5.21s
194:	learn: 0.4644894	total: 1.26s	remaining: 5.2s
195:	learn: 0.4643214	total: 1.27s	remaining: 5.2s
196:	learn: 0.4641812	total: 1.27s	remaining: 5.19s
197:	learn: 0.4640673	total: 1.28s	remaining: 5.18s
198:	learn: 0.463

360:	learn: 0.4442831	total: 2.36s	remaining: 4.17s
361:	learn: 0.4441787	total: 2.36s	remaining: 4.17s
362:	learn: 0.4440240	total: 2.37s	remaining: 4.16s
363:	learn: 0.4439337	total: 2.38s	remaining: 4.15s
364:	learn: 0.4438700	total: 2.38s	remaining: 4.14s
365:	learn: 0.4437274	total: 2.39s	remaining: 4.14s
366:	learn: 0.4436578	total: 2.4s	remaining: 4.13s
367:	learn: 0.4435470	total: 2.4s	remaining: 4.13s
368:	learn: 0.4433940	total: 2.41s	remaining: 4.12s
369:	learn: 0.4433474	total: 2.42s	remaining: 4.11s
370:	learn: 0.4432976	total: 2.42s	remaining: 4.11s
371:	learn: 0.4432379	total: 2.43s	remaining: 4.1s
372:	learn: 0.4431473	total: 2.43s	remaining: 4.09s
373:	learn: 0.4430265	total: 2.44s	remaining: 4.09s
374:	learn: 0.4429863	total: 2.45s	remaining: 4.08s
375:	learn: 0.4429254	total: 2.46s	remaining: 4.08s
376:	learn: 0.4427952	total: 2.46s	remaining: 4.07s
377:	learn: 0.4426932	total: 2.47s	remaining: 4.06s
378:	learn: 0.4425801	total: 2.48s	remaining: 4.06s
379:	learn: 0.4

519:	learn: 0.4268262	total: 3.35s	remaining: 3.09s
520:	learn: 0.4266322	total: 3.35s	remaining: 3.08s
521:	learn: 0.4264789	total: 3.36s	remaining: 3.07s
522:	learn: 0.4264521	total: 3.36s	remaining: 3.07s
523:	learn: 0.4262707	total: 3.37s	remaining: 3.06s
524:	learn: 0.4261536	total: 3.38s	remaining: 3.05s
525:	learn: 0.4259303	total: 3.38s	remaining: 3.05s
526:	learn: 0.4258303	total: 3.39s	remaining: 3.04s
527:	learn: 0.4257220	total: 3.39s	remaining: 3.03s
528:	learn: 0.4256006	total: 3.4s	remaining: 3.03s
529:	learn: 0.4255003	total: 3.4s	remaining: 3.02s
530:	learn: 0.4253867	total: 3.41s	remaining: 3.01s
531:	learn: 0.4252898	total: 3.42s	remaining: 3s
532:	learn: 0.4251654	total: 3.42s	remaining: 3s
533:	learn: 0.4250302	total: 3.43s	remaining: 2.99s
534:	learn: 0.4248640	total: 3.43s	remaining: 2.98s
535:	learn: 0.4247625	total: 3.44s	remaining: 2.98s
536:	learn: 0.4246905	total: 3.44s	remaining: 2.97s
537:	learn: 0.4245788	total: 3.45s	remaining: 2.96s
538:	learn: 0.424555

707:	learn: 0.4053038	total: 4.52s	remaining: 1.86s
708:	learn: 0.4052054	total: 4.53s	remaining: 1.86s
709:	learn: 0.4050922	total: 4.53s	remaining: 1.85s
710:	learn: 0.4049849	total: 4.54s	remaining: 1.84s
711:	learn: 0.4048589	total: 4.54s	remaining: 1.84s
712:	learn: 0.4047493	total: 4.55s	remaining: 1.83s
713:	learn: 0.4046596	total: 4.55s	remaining: 1.82s
714:	learn: 0.4045262	total: 4.56s	remaining: 1.82s
715:	learn: 0.4044238	total: 4.57s	remaining: 1.81s
716:	learn: 0.4042744	total: 4.57s	remaining: 1.8s
717:	learn: 0.4041809	total: 4.58s	remaining: 1.8s
718:	learn: 0.4040677	total: 4.58s	remaining: 1.79s
719:	learn: 0.4039760	total: 4.59s	remaining: 1.78s
720:	learn: 0.4038564	total: 4.59s	remaining: 1.78s
721:	learn: 0.4037346	total: 4.6s	remaining: 1.77s
722:	learn: 0.4035953	total: 4.61s	remaining: 1.76s
723:	learn: 0.4035304	total: 4.61s	remaining: 1.76s
724:	learn: 0.4034390	total: 4.62s	remaining: 1.75s
725:	learn: 0.4033247	total: 4.63s	remaining: 1.75s
726:	learn: 0.4

...

141:	learn: 0.4721470	total: 893ms	remaining: 5.39s
142:	learn: 0.4719992	total: 900ms	remaining: 5.4s
143:	learn: 0.4717733	total: 906ms	remaining: 5.39s
144:	learn: 0.4715867	total: 913ms	remaining: 5.38s
145:	learn: 0.4713634	total: 919ms	remaining: 5.37s
146:	learn: 0.4711626	total: 925ms	remaining: 5.37s
147:	learn: 0.4710050	total: 932ms	remaining: 5.36s
148:	learn: 0.4708102	total: 938ms	remaining: 5.36s
149:	learn: 0.4706491	total: 945ms	remaining: 5.36s
150:	learn: 0.4704654	total: 952ms	remaining: 5.35s
151:	learn: 0.4702981	total: 958ms	remaining: 5.35s
152:	learn: 0.4701691	total: 965ms	remaining: 5.34s
153:	learn: 0.4700504	total: 971ms	remaining: 5.33s
154:	learn: 0.4698959	total: 977ms	remaining: 5.33s
155:	learn: 0.4697332	total: 984ms	remaining: 5.32s
156:	learn: 0.4695292	total: 991ms	remaining: 5.32s
157:	learn: 0.4693109	total: 997ms	remaining: 5.31s
158:	learn: 0.4691426	total: 1s	remaining: 5.3s
159:	learn: 0.4689172	total: 1.01s	remaining: 5.3s
160:	learn: 0.4686

328:	learn: 0.4467035	total: 2.1s	remaining: 4.28s
329:	learn: 0.4466578	total: 2.1s	remaining: 4.27s
330:	learn: 0.4464761	total: 2.11s	remaining: 4.27s
331:	learn: 0.4463608	total: 2.12s	remaining: 4.26s
332:	learn: 0.4462209	total: 2.12s	remaining: 4.25s
333:	learn: 0.4460743	total: 2.13s	remaining: 4.25s
334:	learn: 0.4460255	total: 2.14s	remaining: 4.24s
335:	learn: 0.4459356	total: 2.14s	remaining: 4.23s
336:	learn: 0.4458040	total: 2.15s	remaining: 4.23s
337:	learn: 0.4457649	total: 2.15s	remaining: 4.22s
338:	learn: 0.4456710	total: 2.16s	remaining: 4.21s
339:	learn: 0.4455002	total: 2.17s	remaining: 4.21s
340:	learn: 0.4454392	total: 2.17s	remaining: 4.2s
341:	learn: 0.4453295	total: 2.18s	remaining: 4.2s
342:	learn: 0.4452080	total: 2.19s	remaining: 4.19s
343:	learn: 0.4451055	total: 2.19s	remaining: 4.18s
344:	learn: 0.4449609	total: 2.2s	remaining: 4.18s
345:	learn: 0.4448458	total: 2.21s	remaining: 4.17s
346:	learn: 0.4446498	total: 2.21s	remaining: 4.16s
347:	learn: 0.444

517:	learn: 0.4254404	total: 3.29s	remaining: 3.06s
518:	learn: 0.4253202	total: 3.3s	remaining: 3.06s
519:	learn: 0.4252117	total: 3.31s	remaining: 3.05s
520:	learn: 0.4251387	total: 3.31s	remaining: 3.04s
521:	learn: 0.4250372	total: 3.32s	remaining: 3.04s
522:	learn: 0.4248848	total: 3.33s	remaining: 3.03s
523:	learn: 0.4247142	total: 3.33s	remaining: 3.02s
524:	learn: 0.4246889	total: 3.34s	remaining: 3.02s
525:	learn: 0.4245996	total: 3.34s	remaining: 3.01s
526:	learn: 0.4245165	total: 3.35s	remaining: 3.01s
527:	learn: 0.4243881	total: 3.36s	remaining: 3s
528:	learn: 0.4242568	total: 3.36s	remaining: 2.99s
529:	learn: 0.4242195	total: 3.37s	remaining: 2.99s
530:	learn: 0.4241275	total: 3.38s	remaining: 2.98s
531:	learn: 0.4240038	total: 3.38s	remaining: 2.98s
532:	learn: 0.4239312	total: 3.39s	remaining: 2.97s
533:	learn: 0.4239063	total: 3.4s	remaining: 2.96s
534:	learn: 0.4237776	total: 3.4s	remaining: 2.96s
535:	learn: 0.4236530	total: 3.41s	remaining: 2.95s
536:	learn: 0.4235

705:	learn: 0.4059152	total: 4.47s	remaining: 1.86s
706:	learn: 0.4057618	total: 4.48s	remaining: 1.86s
707:	learn: 0.4056812	total: 4.49s	remaining: 1.85s
708:	learn: 0.4055953	total: 4.49s	remaining: 1.84s
709:	learn: 0.4054778	total: 4.5s	remaining: 1.84s
710:	learn: 0.4053410	total: 4.5s	remaining: 1.83s
711:	learn: 0.4052434	total: 4.51s	remaining: 1.82s
712:	learn: 0.4051062	total: 4.52s	remaining: 1.82s
713:	learn: 0.4050214	total: 4.52s	remaining: 1.81s
714:	learn: 0.4049386	total: 4.53s	remaining: 1.81s
715:	learn: 0.4048179	total: 4.54s	remaining: 1.8s
716:	learn: 0.4047622	total: 4.54s	remaining: 1.79s
717:	learn: 0.4046406	total: 4.55s	remaining: 1.79s
718:	learn: 0.4045036	total: 4.56s	remaining: 1.78s
719:	learn: 0.4044268	total: 4.56s	remaining: 1.77s
720:	learn: 0.4042925	total: 4.57s	remaining: 1.77s
721:	learn: 0.4041889	total: 4.57s	remaining: 1.76s
722:	learn: 0.4041131	total: 4.58s	remaining: 1.75s
723:	learn: 0.4040461	total: 4.58s	remaining: 1.75s
724:	learn: 0.4

890:	learn: 0.3886444	total: 5.65s	remaining: 691ms
891:	learn: 0.3885842	total: 5.66s	remaining: 685ms
892:	learn: 0.3884538	total: 5.66s	remaining: 679ms
893:	learn: 0.3883746	total: 5.67s	remaining: 672ms
894:	learn: 0.3883305	total: 5.68s	remaining: 666ms
895:	learn: 0.3883112	total: 5.68s	remaining: 660ms
896:	learn: 0.3882542	total: 5.69s	remaining: 653ms
897:	learn: 0.3881470	total: 5.7s	remaining: 647ms
898:	learn: 0.3880548	total: 5.7s	remaining: 640ms
899:	learn: 0.3879486	total: 5.71s	remaining: 634ms
900:	learn: 0.3878750	total: 5.71s	remaining: 628ms
901:	learn: 0.3877785	total: 5.72s	remaining: 621ms
902:	learn: 0.3876697	total: 5.72s	remaining: 615ms
903:	learn: 0.3875122	total: 5.73s	remaining: 609ms
904:	learn: 0.3874325	total: 5.74s	remaining: 602ms
905:	learn: 0.3873313	total: 5.74s	remaining: 596ms
906:	learn: 0.3872599	total: 5.75s	remaining: 590ms
907:	learn: 0.3871469	total: 5.76s	remaining: 583ms
908:	learn: 0.3869782	total: 5.76s	remaining: 577ms
909:	learn: 0.

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe81 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', LGBMClassifier())
    ])
    pipe81.fit(X_train, y_train)
    y_predicted=pipe81.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
roc = []
for _ in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=.3, random_state=0) 
    
    pipe7 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('sampling', SMOTE()),
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe7.fit(X_train, y_train)
    y_predicted=pipe7.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
#     recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe81 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', XGBClassifier())
    ])
    pipe81.fit(X_train, y_train)
    y_predicted=pipe81.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe81.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
print(np.mean(roc))

In [ ]:
X_sms = KNNImputer().fit_transform(X)
X_sms

In [6]:
X_sms = SimpleImputer(strategy='mean').fit_transform(X)

In [7]:
X_sms = pd.DataFrame(X_sms)
X_sms.columns = columns

In [8]:
X_sms.isna().sum()

age                                                 0
number_dependent_family_members                     0
monthly_income                                      0
number_of_credit_lines                              0
real_estate_loans                                   0
ratio_debt_payment_to_income                        0
credit_line_utilization                             0
number_of_previous_late_payments_up_to_59_days      0
number_of_previous_late_payments_up_to_89_days      0
number_of_previous_late_payments_90_days_or_more    0
dtype: int64

In [17]:
over = SMOTE(sampling_strategy=.1)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeli = pipeline.Pipeline(steps=steps)

In [ ]:
?SMOTE

In [18]:
X_smt, y_smt = pipeli.fit_resample(X_sms, y)
Counter(y_smt)

Counter({0: 6714, 1: 6714})

In [19]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    pipea4 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipea4.fit(X_train, y_train)
    y_predicted=pipea4.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.8696081149134003


In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    pipea5 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', CatBoostClassifier())
    ])
    pipea5.fit(X_train, y_train)
    y_predicted=pipea5.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe9 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe9.fit(X_train, y_train)
    y_predicted=pipe9.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe9.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe10 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer(n_neighbors=20)),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', VotingClassifier(estimators=[
            ('forest50', RandomForestClassifier(n_estimators=50)),
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe10.fit(X_train, y_train)
    y_predicted=pipe10.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe10.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

## PCA GridSearch PolynomialFeature

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
# from sklearn.feature_selection import SelectKBest

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    pipe12 = Pipeline(steps=[
            ('preprocessing', ColumnTransformer(transformers=[
                ('numeric', Pipeline(steps=[
                    ('impute', SimpleImputer(strategy='mean')),
                    ('scale', StandardScaler())
                ]), columns),
            ])),
            ('classifier', BaggingClassifier(base_estimator=DecisionTreeClassifier()))
        ])
    params = {
        'classifier__n_estimators': [80, 100, 120],
    }

    RF_gs = GridSearchCV(pipe12, param_grid=params, scoring='roc_auc')
    RF_gs.fit(X_train, y_train)
    y_predicted=RF_gs.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:,1]))
print(np.mean(roc))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=RF_gs.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe14 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', StandardScaler()),
                ('poly', PolynomialFeatures())
            ]), columns),
        ])),
        ('classifier', VotingClassifier(estimators=[
            ('forest50', RandomForestClassifier(n_estimators=50)),
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe14.fit(X_train, y_train)
    y_predicted=pipe14.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe14.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe15 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler()),
            ]), columns),
        ])),
        ('umap', UMAP(n_components=8)),
        ('classifier',RandomForestClassifier())
    ])
    pipe15.fit(X_train, y_train)
    y_predicted=pipe15.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe15.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
from umap import UMAP

In [ ]:
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns
# X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_rus)

In [ ]:
umap_reducer = umap.UMAP(n_components=2)
train_embedding = umap_reducer.fit_transform(X_imputed[columns])
plt.figure(figsize=(14, 6))
plt.scatter(train_embedding[:, 0], train_embedding[:, 1], s=4, c=y_rus)
plt.show()

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns
iso = IsolationForest()
yhat = iso.fit_predict(X_imputed)
mask = yhat != -1
X_iso, y_iso = X_imputed[mask], y_rus[mask]

In [ ]:
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns

In [ ]:
iso = IsolationForest()
ocs = OneClassSVM()
y_iso = iso.fit_predict(X_imputed)
y_ocs = ocs.fit_predict(X_imputed)

m_iso = y_iso != -1
m_ocs = y_ocs != -1

In [ ]:
X_iso, y_iso = X_imputed[m_iso], y_rus[m_iso]
X_ocs, y_ocs = X_imputed[m_ocs], y_rus[m_ocs]

In [ ]:
X_iso.shape

In [ ]:
y_iso.shape

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_iso, y_iso):
    X_train, X_test = X_iso.iloc[train_idx], X_iso.iloc[test_idx]
    y_train, y_test = y_iso.iloc[train_idx], y_iso.iloc[test_idx]
    
    pipe20 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier())
    ])
    pipe20.fit(X_train, y_train)
    y_predicted=pipe20.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_iso, y_iso):
    X_train, X_test = X_iso.iloc[train_idx], X_iso.iloc[test_idx]
    y_train, y_test = y_iso.iloc[train_idx], y_iso.iloc[test_idx]
    
    pipe22 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=30))
    ])
    pipe22.fit(X_train, y_train)
    y_predicted=pipe22.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe22.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = pd.concat([X_rus, y_rus], axis=1).corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
?df.corr()

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    pipe30 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', XGBClassifier())
    ])
    pipe30.fit(X_train, y_train)
    y_predicted=pipe30.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

 ### ClassifierChain || pipe model series 40

In [ ]:
from eli5 import show_weights
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import cross_val_score

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe40 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (LGBMClassifier()))
    ])
    pipe40.fit(X_train, y_train)
    y_predicted=pipe40.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe41 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (XGBClassifier()))
    ])
    pipe41.fit(X_train, y_train)
    y_predicted=pipe41.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe40 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=20))
    ])
    pipe40.fit(X_train, y_train)
    y_predicted=pipe40.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe42 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (CatBoostClassifier()))
    ])
    pipe42.fit(X_train, y_train)
    y_predicted=pipe42.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe43 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', GradientBoostingClassifier(n_estimators=200, learning_rate=0.5,max_depth=1, random_state=0))
    ])
    pipe43.fit(X_train, y_train)
    y_predicted=pipe43.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
cv = KFold(n_splits=8, random_state=1, shuffle=True)
# create model
# model = DecisionTreeClassifier()
# evaluate model
scores = cross_val_score(pipe9, X_rus, y_rus, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy:',np.mean(scores))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe43.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

### Dimensionality reduction || pipe version 50

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(SimpleImputer(strategy='mean').fit_transform(X_rus))

In [ ]:
cv = StratifiedKFold(n_splits=8, random_state=1, shuffle=True)
# create model
model = DecisionTreeClassifier()
# evaluate model
scores = cross_val_score(pipe9, X_rus, y_rus, scoring='roc_auc', cv=cv, n_jobs=-1)
# report performance
print('Accuracy:',np.mean(scores))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_pca, y_rus):
    X_train, X_test = X_pca[train_idx], X_pca[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe40 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (LGBMClassifier()))
    ])
    pipe40.fit(X_train, y_train)
    y_predicted=pipe40.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
test = pd.read_csv('../input/iba-ml1-mid-project/test.csv')
test['credit_line_utilization'] = test['credit_line_utilization'].str.replace(',', '.').astype(float)
test_prediction = pipe9z.predict(test.iloc[:, 1:])
submission = pd.DataFrame({'Predicted':test_prediction})
submission = submission.set_index(pd.Index(np.arange(1, 48109))).reset_index()

In [ ]:
submission.to_csv('submission9z.csv')

In [ ]:
perm = PermutationImportance(pipe40, scoring = 'roc_auc'
,random_state=101).fit(X_test, y_test)
show_weights(perm, feature_names = list(X_test.columns))

### Outlier removed || Pipe 60 

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
from sklearn.ensemble import IsolationForest
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns
iso = IsolationForest()
yhat = iso.fit_predict(X_imputed)
mask = yhat != -1
X_iso, y_iso = X_imputed[mask], y[mask]

In [ ]:
print(X_imputed.shape)
print(X_iso.shape)

In [ ]:
Counter(y_iso)

In [ ]:
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = imblearn.pipeline.Pipeline(steps=steps)
X_smt, y_smt = pipeline.fit_resample(X_iso, y_iso)
Counter(y_smt)

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    pipe60 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier())
    ])
    pipe60.fit(X_train, y_train)
    y_predicted=pipe60.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe60.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    pipe61 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', VotingClassifier(estimators=[
            ('forest50', RandomForestClassifier(n_estimators=50)),
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe61.fit(X_train, y_train)
    y_predicted=pipe61.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe61.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    param_grid = {
        'pca__n_components': [2, 4, 5, 9],
        'classifier__n_estimators': [80, 100, 150, 200, 300],
    }
    pipe62 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('pca', PCA()),
        ('classifier', RandomForestClassifier())
    ])
    search = GridSearchCV(pipe62, param_grid, n_jobs=-1)
    search.fit(X_train, y_train)
#     pipe62.fit(X_train, y_train)
    y_predicted=search.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=search.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
roc = []
recall = []
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus)

param_grid = {
#     'classifier__n_estimators': [100, 200],
#     'tfidf__norm': ['l1','l2']
}
pipe63 = Pipeline(steps=[
    ('preprocessing', ColumnTransformer(transformers=[
        ('numeric', Pipeline(steps=[
            ('impute', KNNImputer()),
            ('scale', StandardScaler())
        ]), columns),
    ])),
#     ('tfidf', TfidfTransformer(norm='l1')),
    ('classifier', RandomForestClassifier(n_estimators=200))
])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
search1 = GridSearchCV(pipe63, param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
search1.fit(X_train, y_train)
#     pipe62.fit(X_train, y_train)
y_predicted=search1.predict(X_test)
print(roc_auc_score(y_test, y_predicted))
# recall.append(recall_score(y_test, y_predicted))
    
# print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=search1.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
roc = []
recall = []
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus)

param_grid = {
#     'classifier__n_estimators': [100, 200],
#     'tfidf__norm': ['l1','l2']
    'pca__n_components': [1, 5]
}
pipe64= Pipeline(steps=[
    ('preprocessing', ColumnTransformer(transformers=[
        ('numeric', Pipeline(steps=[
            ('impute', IterativeImputer()),
            ('scale', RobustScaler())
        ]), columns),
    ])),
    ('pca', PCA()),
    ('classifier', RandomForestClassifier(n_estimators=200))
])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
search2 = GridSearchCV(pipe64, param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
search2.fit(X_train, y_train)
#     pipe62.fit(X_train, y_train)
y_predicted=search2.predict(X_test)
print(roc_auc_score(y_test, y_predicted))
# recall.append(recall_score(y_test, y_predicted))
    
# print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=search2.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
?PCA

In [ ]:
roc = []
recall = []
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus)

param_grid = {
    'classifier__n_estimators': [100,150],
    'pca__n_components': [6, 9],
#     'classifier__learning_rate': [0.01, 0.2, 1],
#     "classifier__min_samples_split": np.linspace(0.1, 0.5, 12),
#     "classifier__min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "preprocessing__numeric__impute__n_neighbors": [10, 15]   
}

pipe65= Pipeline(steps=[
    ('preprocessing', ColumnTransformer(transformers=[
        ('numeric', Pipeline(steps=[
            ('impute', KNNImputer()),
            ('scale', StandardScaler())
        ]), columns),
    ])),
    ('pca', PCA()),
    ('classifier', RandomForestClassifier())
])
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
search3 = GridSearchCV(pipe65, param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
search3.fit(X_train, y_train)
#     pipe62.fit(X_train, y_train)
y_predicted=search3.predict(X_test)
print(roc_auc_score(y_test, y_predicted))
# recall.append(recall_score(y_test, y_predicted))

# print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=search3.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
search3.best_params_

In [ ]:
Counter(y_iso)

In [ ]:
ros = RandomOverSampler(sampling_strategy=0.5)
rus = RandomUnderSampler(sampling_strategy=0.8)

X_ros, y_ros = ros.fit_resample(X, y)
X_comb, y_comb = rus.fit_resample(X_ros, y_ros)

In [ ]:
Counter(y_ros)

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe65 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe65.fit(X_train, y_train)
    y_predicted=pipe65.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    
print(np.mean(roc))

In [ ]:
Counter(y_rus)

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe65.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [31]:
test = pd.read_csv('../input/iba-ml1-mid-project/test.csv')
test['credit_line_utilization'] = test['credit_line_utilization'].str.replace(',', '.').astype(float)
test_prediction = pipe82.predict_proba(test.iloc[:, 1:])
submission = pd.concat([ test['Id'], pd.DataFrame({'Predicted':test_prediction[:, 1]})], axis=1)

In [32]:
submission.to_csv('submission82.csv', index=False)

In [ ]:
submission

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
!kaggle competitions submit -c iba-ml1-mid-project -f submission7n.csv -m "Message"